# Modulation Index analysis 

## load the basic settings

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import mne_connectivity
import numpy as np
#from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance

import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions, plotting_parameters
#import src.analysis.utils.general_utils as utils # import utils functions one by one by name
from src.analysis.power.power_traces import make_multi_channel_evokeds_for_all_conditions_and_rois, plot_power_traces_for_all_rois

['c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg', '', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\LuoRuoxi\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


In [2]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task='GlobalLocal', root=LAB_root)
save_dir_root = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

In [ ]:
#subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057'] # use one subject for testing

## load the data

In [7]:
print(save_dir_root)
epochs = mne.read_epochs(os.path.join(layout.root, 'derivatives', 'freqFilt','figs', 'D0057', 'D0057_Stimulus_ev1-epo.fif'), preload=True)

C:\Users\LuoRuoxi\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs
Reading C:\Users\LuoRuoxi\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated


In [ ]:
rois_dict = {
    'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}